# Helpful Reviews

### Data Wrangling

### Corey J Wade, WM^3

## Introduction

This Jupyter Notebook will present data wrangling on the Amazon Book Reviews dataset courtesy of Julian McAuley, UCSC: 

http://jmcauley.ucsd.edu/data/amazon/

Our goal is to create a new metric, Helpful Rating, that can be generated immediately after a review is written.

## Open Dataset

In [2]:
# import pandas and numpy
import pandas as pd
import numpy as np

# Open DataFrame
df = pd.read_csv('Amazon_Data_Frame.csv')

# View DataFrame
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,000100039X,"[0, 0]",5.0,Spiritually and mentally inspiring! A book tha...,"12 16, 2012",A10000012B7CGYKOMPQ4L,Adam,Wonderful!,1355616000
1,1,000100039X,"[0, 2]",5.0,This is one my must have books. It is a master...,"12 11, 2003",A2S166WSCFIFP5,"adead_poet@hotmail.com ""adead_poet@hotmail.com""",close to god,1071100800
2,2,000100039X,"[0, 0]",5.0,This book provides a reflection that you can a...,"01 18, 2014",A1BM81XB4QHOA3,"Ahoro Blethends ""Seriously""",Must Read for Life Afficianados,1390003200
3,3,000100039X,"[0, 0]",5.0,I first read THE PROPHET in college back in th...,"09 27, 2011",A1MOSTXNIO5MPJ,Alan Krug,Timeless for every good and bad time in your l...,1317081600
4,4,000100039X,"[7, 9]",5.0,A timeless classic. It is a very demanding an...,"10 7, 2002",A2XQ5LZHTD4AFT,Alaturka,A Modern Rumi,1033948800


## Delete Unnamed Column

In [3]:
# Delete extraneous column
del df['Unnamed: 0']

## Column Info

In [4]:
# Get info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8898041 entries, 0 to 8898040
Data columns (total 9 columns):
asin              object
helpful           object
overall           float64
reviewText        object
reviewTime        object
reviewerID        object
reviewerName      object
summary           object
unixReviewTime    int64
dtypes: float64(1), int64(1), object(7)
memory usage: 611.0+ MB


Note that the number of non-null objects for each column are not shown for a dataset this large.

## Delete NaN Reviews

In [7]:
# Count number of reviews with no text
df['reviewText'].isnull().sum()

591

In [8]:
# Delete reviews with no text
df = df.dropna(axis=0, subset=['reviewText'])

In [9]:
# Check that rows have been deleted
df['reviewText'].isnull().sum()

0

## Convert Helpful List

In [10]:
# Convert 'helpful' column to list
import ast
helpful_List = np.array(df.helpful.apply(lambda row: ast.literal_eval(row)).tolist())

# Separate list into distinct columns
df['Helpful_Votes'] = helpful_List[:,0]
df['Total_Votes'] = helpful_List[:,1]

## Create Helpful_Percentage

In [ ]:
# Create column for helpful percentage, convert NaN values to 0
df['Helpful_Percentage'] = (df['Helpful_Votes']/df['Total_Votes']).fillna(0)

In [11]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Helpful_Votes,Total_Votes,Helpful_Percentage
0,000100039X,"[0, 0]",5.0,Spiritually and mentally inspiring! A book tha...,"12 16, 2012",A10000012B7CGYKOMPQ4L,Adam,Wonderful!,1355616000,0,0,0.000000
1,000100039X,"[0, 2]",5.0,This is one my must have books. It is a master...,"12 11, 2003",A2S166WSCFIFP5,"adead_poet@hotmail.com ""adead_poet@hotmail.com""",close to god,1071100800,0,2,0.000000
2,000100039X,"[0, 0]",5.0,This book provides a reflection that you can a...,"01 18, 2014",A1BM81XB4QHOA3,"Ahoro Blethends ""Seriously""",Must Read for Life Afficianados,1390003200,0,0,0.000000
3,000100039X,"[0, 0]",5.0,I first read THE PROPHET in college back in th...,"09 27, 2011",A1MOSTXNIO5MPJ,Alan Krug,Timeless for every good and bad time in your l...,1317081600,0,0,0.000000
4,000100039X,"[7, 9]",5.0,A timeless classic. It is a very demanding an...,"10 7, 2002",A2XQ5LZHTD4AFT,Alaturka,A Modern Rumi,1033948800,7,9,0.777778


#### Error Check

Check for Helpful Percentage above 0. 

In [14]:
len(df[df['Helpful_Percentage']>1.0])

15

#### Drop Errors

In [15]:
df = df.drop(df[df['Helpful_Percentage']>1.0].index)

In [16]:
len(df[df['Helpful_Percentage']>1.0])

0

## Delete No Vote Rows

We are only interested in reviews that received a thumbs up or thumbs down since they can be trained and tested. If the review contains no votes, it should be eliminated.

In [17]:
# Find number of rows with 0 votes
len(df[df['Total_Votes']==0])

4141097

In [18]:
# Copy original dataframe
df_original = df

# Drop rows with 0 votes from data frame
df = df.drop(df[df['Total_Votes']==0].index)

In [19]:
# Check drop
len(df[df['Total_Votes']==0])

0

## Add Columns Analyzing Text

Even though words in a bag, sentiment analysis, and/or other approved text analysis may be applied later, I want to add a couple of basic relevant columns.

In [120]:
# import regular expressions, string
import re
import string

def average_sentence_length(text):
    # choose endings to split sentences with re
    sentence_endings = r"[.?!]"
    # use spaces to separate words with re
    spaces = r"\s+"
    
    # split sentences
    sentences = re.split(sentence_endings, text)
    # filter out blanks (occurs after last punctuation mark)
    sentences = list(filter(None, sentences))
    
    count = 0.0
    for sentence in sentences:
        # split sentence into individual words
        words = re.split(spaces, sentence)
        # filter out empty words
        words = list(filter(None, words))
        # add word count
        count += len(words)
    return count/len(sentences)


def average_word_length(text):
    
    # prepare punctuation strip with re
    strip = re.compile('[%s]' % re.escape(string.punctuation))
    
    # remove punctuation from text
    words = strip.sub('', text)

    # use spaces to separate words with re
    spaces = r"\s+"
    
    # split sentences into list of words
    words = re.split(spaces, words)
    
    count = 0
    for word in words:
        # count length of each word
        count += len(word)
    return count/len(words)

In [121]:
# Create column for length of review
df['Review_Length'] = df['reviewText'].str.len()

# Create column for average length of sentence
df['Sentence_Length'] = df['reviewText'].apply(average_sentence_length)

# Create column for average word length
df['Word_Length'] = df['reviewText'].apply(average_word_length)

## Create Target Column: Helpful Rating

There is no column to measure how helpful a particular review is. Creating such a column is essential because it will be our target variable.

I start by creating a new function to scale Helpful_Votes from (0,1). I choose a piecewise linear function to create a desirable spread.

#### Helpful_Votes_Scaled 

In [133]:
# determine slope, y_intercept from two given points
def two_points_line(a,b):
    slope = (b[1] - a[1])/(b[0]-a[0])
    y_int = a[1] - slope * a[0]
    return slope, y_int

In [135]:
# given input of helpful_votes, convert output to score between 0 and 1
def helpful_votes_rating(x):
    m1, b1 = two_points_line((0,0), (5,0.4))
    if x < 6: 
        y = m1 * x + b1
    elif x < 16:
        m2, b2 = two_points_line((5,0.4), (15,0.75))
        y = m2 * x + b2
    elif x < 26: 
        m3, b3 = two_points_line((15,0.75), (25,0.9))
        y = m3 * x + b3
    elif x < 51: 
        m4, b4 = two_points_line((25, 0.9), (50, 0.95))
        y = m4 * x + b4
    elif x < 101: 
        m5, b5 = two_points_line((50, 0.95), (100, 0.99))
        y = m5 * x + b5
    elif x < 251: 
        m6, b6 = two_points_line((100, 0.99), (250, 0.999))
        y = m6 * x + b6
    elif x < 1000: 
        m6, b6 = two_points_line((250, 0.999), (999, 0.999999))
        y = m6 * x + b6
    else: 
        y = 1.0
    return y

In [136]:
# create new column converting Helpful_Votes to a rating
Helpful_Votes_Scaled = df['Helpful_Votes'].apply(helpful_votes_rating)

#### Helpful_Percentage_Book_Score

A review that receives more helpful votes because the book is popular is not necessarily more helpful than reviews for books that are less popular. In addition to Helpful_Votes_Scaled, and Helpful_Percentage, I add one last piece, Helpful_Percentage_Book_Score. The purpose of this series is to acknowledge reviews that have accrued the majority of helpful votes per book.

In [137]:
# Create column that sums helpful votes per book
df['Helpful_Votes_Book'] = df.groupby('asin')['Helpful_Votes'].transform('sum')

# Create series that de
Helpful_Percentage_Book_Score = (df['Helpful_Votes']/df['Helpful_Votes_Book']).fillna(0)

# Delete extraneous column
del df['Helpful_Votes_Book']

# Show new series
Helpful_Percentage_Book_Score.describe()

count    4.756338e+06
mean     7.477917e-02
std      1.397735e-01
min      0.000000e+00
25%      3.267974e-03
50%      1.776650e-02
75%      7.692308e-02
max      1.000000e+00
dtype: float64

Note that the new series is not scaled efficiently. Since it's a right skewed distribution, I will take the log.

After some trial and error, using e-1 as a guide, I came up with the following formula.

In [138]:
# Adjusted formula for Helpful_Percentage_Book_Score
Helpful_Percentage_Book_Score = np.log(Helpful_Percentage_Book_Score+1.71828)

# Show updated series
Helpful_Percentage_Book_Score.describe()

count    4.756338e+06
mean     5.812972e-01
std      6.996605e-02
min      5.413238e-01
25%      5.432239e-01
50%      5.516104e-01
75%      5.851182e-01
max      9.999993e-01
dtype: float64

This log distribution ensures that reviewers are not penalized if they do not have a large percentage of the total helpful reviews, while reviewers whose percentage of helpful reviews are a large percentage of the total are rewarded.

#### Helpful_Rating Formula

In [139]:
# Create weighted helpful score
df['Helpful_Rating'] = 0.36*Helpful_Votes_Scaled + 0.60*df['Helpful_Percentage'] + 0.04*Helpful_Percentage_Book_Score

I spent some time checking tables and tweaking the weights. I made additional adjustments after deciding that I would train/ test on a smaller subset with a minimum number of total reviews.

In [153]:
# Narrow columns, reset index
dfr = df[['overall', 'reviewText', 'Helpful_Votes', 'Total_Votes', 'Helpful_Percentage', 'Helpful_Rating', 'Review_Length', 'Sentence_Length', 'Word_Length']].reset_index(drop=True)

#### Display Results

In [154]:
# Display dataframe with top helpful scores.
dfr.sort_values(by = ['Helpful_Rating'], ascending=False)

,overall,reviewText,Helpful_Votes,Total_Votes,Helpful_Percentage,Helpful_Rating,Review_Length,Sentence_Length,Word_Length
866465,5.0,After receiving this textbook for my third lev...,472,472,1.000000,0.999320,3969,27.772727,5.290164
3946146,5.0,I looked through a lot of other planners while...,239,239,1.000000,0.999220,1507,16.000000,4.319703
3986461,5.0,This is exactly what I was looking for. A thre...,195,195,1.000000,0.998228,1567,17.823529,4.096346
2899857,1.0,"""Trapped under a beam with the countdown ticki...",3895,3908,0.996673,0.997685,7675,13.475728,4.415680
3939301,5.0,It's not your normal text/workbook. You have t...,426,427,0.997658,0.997401,3000,13.268293,4.415730
2384165,5.0,Since the listing doesn't say which books are ...,308,309,0.996764,0.997398,284,3.666667,5.068182
295129,4.0,"This review is for the ""Second Edition of the ...",279,280,0.996429,0.997200,3167,13.800000,4.673546
3162573,5.0,Ten percent of the books I have reviewed here ...,154,154,1.000000,0.997011,2463,25.312500,4.982278
2479965,5.0,I hate math. Memories of trying to understand ...,232,232,1.000000,0.996978,2431,18.080000,4.275168
2541565,5.0,"""SHARPE'S EAGLE wasn't written in a hurry. I h...",153,153,1.000000,0.996719,2034,20.764706,4.647564


## Splits

The more votes a review receives, the more likely the training data will be accurate. Currently all votes in df have at least 1 total vote. The following table gives an indication of the breakdown.

In [155]:
dfr.describe()

,overall,Helpful_Votes,Total_Votes,Helpful_Percentage,Helpful_Rating,Review_Length,Sentence_Length,Word_Length
count,4.756338e+06,4.756338e+06,4.756338e+06,4.756338e+06,4.756338e+06,4.756338e+06,4.756338e+06,4.756338e+06
mean,4.101044e+00,5.625667e+00,7.575643e+00,7.348979e-01,5.502079e-01,1.036185e+03,1.623846e+01,4.454504e+00
std,1.202299e+00,2.663631e+01,3.052654e+01,3.426306e-01,2.448466e-01,1.182085e+03,6.536170e+00,4.557439e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.165295e-02,1.000000e+00,5.000000e-02,0.000000e+00
25%,4.000000e+00,1.000000e+00,1.000000e+00,5.000000e-01,3.923671e-01,2.820000e+02,1.196154e+01,4.185185e+00
50%,5.000000e+00,2.000000e+00,2.000000e+00,9.333333e-01,6.506276e-01,6.410000e+02,1.550000e+01,4.443536e+00
75%,5.000000e+00,4.000000e+00,6.000000e+00,1.000000e+00,6.819020e-01,1.372000e+03,1.960000e+01,4.709677e+00
max,5.000000e+00,2.331100e+04,2.421200e+04,1.000000e+00,9.993204e-01,3.265800e+04,5.910000e+02,2.559412e+02


The focus is currently on the Total_Votes column. Note that half of the reivews only have 2 total votes.

In [156]:
# Show number of rows in dataframe
len(dfr)

4756338

In [157]:
# Show number of rows with more than 1 vote
len(dfr[dfr['Total_Votes']>1])

3106418

In [158]:
# Show number of rows with more than 2 votes
len(dfr[dfr['Total_Votes']>2])

2299696

In [159]:
# Show number of rows with more than 5 votes
len(dfr[dfr['Total_Votes']>5])

1281080

In [160]:
# Show number of rows with more than 9 votes
len(dfr[dfr['Total_Votes']>9])

787981

In [161]:
# Show number of rows with more than 7 votes
len(dfr[dfr['Total_Votes']>7])

979545

In [162]:
# Show number of rows with more than 7 votes
len(dfr[dfr['Total_Votes']>6])

1113162

In [163]:
# Create new dataframes based on results above
df_3 = dfr[dfr['Total_Votes']>2]
df_7 = dfr[dfr['Total_Votes']>6]
df_10 = dfr[dfr['Total_Votes']>9]

I will choose df_7 for now. The idea is that only reviews with 7 or more votes will be considered on our initial training and test data. The dataset still contains over 1 million reviews.

In [164]:
df_7.head(20)

,overall,reviewText,Helpful_Votes,Total_Votes,Helpful_Percentage,Helpful_Rating,Review_Length,Sentence_Length,Word_Length
1,5.0,A timeless classic. It is a very demanding an...,7,9,0.777778,0.657681,813,13.818182,4.260000
9,2.0,I read this about a year ago and can't recall ...,2,9,0.222222,0.212633,711,16.750000,4.194030
14,5.0,This is one of the first (literary) books I re...,81,92,0.880435,0.902674,1542,15.000000,4.258865
18,5.0,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000,0.683638,2294,22.705882,4.883289
19,5.0,Gibran Khalil Gibran was born in 1883 in what ...,8,10,0.800000,0.683638,712,19.428571,4.103704
34,5.0,This book is almost as if Kahlil Gibran took t...,5,8,0.625000,0.540768,531,24.000000,4.395833
35,5.0,"Certainly the words are of Kahlil Gibran, but ...",10,12,0.833333,0.728884,700,21.000000,4.675000
36,2.0,"I evidently misread the writeup, I thought it ...",0,13,0.000000,0.021653,178,11.666667,3.885714
39,1.0,As Amin Rihani described his own friend Gibran...,2,9,0.222222,0.212633,250,10.000000,4.875000
41,5.0,The Prophet dispenses ultimate wisdom to his l...,8,10,0.800000,0.683638,528,14.714286,4.089109


In [165]:
# Show info of df_7
df_7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1113162 entries, 1 to 4756323
Data columns (total 9 columns):
overall               1113162 non-null float64
reviewText            1113162 non-null object
Helpful_Votes         1113162 non-null int64
Total_Votes           1113162 non-null int64
Helpful_Percentage    1113162 non-null float64
Helpful_Rating        1113162 non-null float64
Review_Length         1113162 non-null int64
Sentence_Length       1113162 non-null float64
Word_Length           1113162 non-null float64
dtypes: float64(5), int64(3), object(1)
memory usage: 84.9+ MB


## Save files

In [167]:
df_7.to_csv('df_7.csv')